## Desarrollo con el método SURF

SURF es un algoritmo para la representación y la comparación local de imágenes. De manera similar a otros métetodos también basados en descriptores locales, los *keyponits* de una imagen dada se definen como características sobresalientes de una representación invariante de escala. Dicho análisis se obtiene mediante la convolución de la imagen inicial con granos discretos a varias escalas (filtros de caja) seguido del uso de gradientes locales (intensidad y orientación).


### Carga de bibliotecas


In [ ]:
!pip uninstall opencv-python -y

!pip install opencv-contrib-python==3.4.2.17 --force-reinstall

Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 141kB/s 
     |████████████████████████████████| 14.5MB 323kB/s 
ERROR: imgaug 0.2.9 requires opencv-python, which is not installed.
ERROR: dopamine-rl 1.0.5 requires opencv-python>=3.4.1.15, which is not installed.
ERROR: albumentations 0.1.12 requires opencv-python, which is not installed.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:


### Carga de bibliotecas


In [ ]:
import cv2
import numpy as np
import os

### Lectura del dataset (dentro del entorno colab) y Aplicación de SURF

In [ ]:


train_path = 'Animales/Training'

training_names = os.listdir(train_path)


image_paths = []
#Etiquetas
image_classes = []

class_id = 0


def imglist(path):
    return [os.path.join(path,f) for f in os.listdir(path)]


for training_name in training_names:
    dir = os.path.join(train_path, training_name)
    class_path = imglist(dir)
    image_paths += class_path
    image_classes += [class_id]*len(class_path)
    class_id += 1

#Vector vacio de características (Todas las imágenes)
caractSURF = []

for image_path in image_paths:
    im = cv2.imread(image_path)
    gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
    surf = cv2.xfeatures2d.SURF_create()
    kp, des = surf.detectAndCompute(gray,None)
    caractSURF.append((image_path,des))


descriptors = caractSURF[0][1]
for image_path, descriptor in caractSURF[1:]:
    descriptors = np.vstack((descriptors, descriptor))

descriptors = descriptors.astype(float)


### Bag of words

In [ ]:
#Kmeans

from scipy.cluster.vq import kmeans ,vq

k = 60

voc,variance = kmeans(descriptors,k,1)

im_features = np.zeros((len(image_paths),k), "float32")

for i in range(len(image_paths)):
    words, distance = vq(caractSURF[i][1],voc)
    for w in words:
        im_features[i][w] += 1


### Agregar columna de clases y creación de Dataframe

In [ ]:
y = np.array(image_classes)

x = np.array(im_features)

import pandas as pd

Data = pd.DataFrame(x) 

#60 Etiqueta

Data['60'] = y


Data = Data.rename(columns = {'60':'Clase'})


Data.to_csv('SURF.csv', header = True, index= True)

In [ ]:
Data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,Clase
0,44.0,7.0,117.0,24.0,19.0,39.0,27.0,132.0,44.0,108.0,84.0,120.0,48.0,39.0,62.0,47.0,38.0,32.0,15.0,39.0,37.0,96.0,121.0,34.0,18.0,32.0,40.0,101.0,117.0,26.0,29.0,42.0,18.0,85.0,66.0,50.0,92.0,13.0,13.0,54.0,37.0,26.0,29.0,44.0,69.0,78.0,6.0,24.0,60.0,28.0,30.0,62.0,354.0,94.0,60.0,102.0,37.0,100.0,14.0,64.0,0
1,204.0,18.0,87.0,12.0,27.0,67.0,44.0,178.0,84.0,140.0,98.0,94.0,63.0,19.0,30.0,83.0,20.0,15.0,45.0,50.0,119.0,120.0,195.0,32.0,53.0,73.0,69.0,304.0,133.0,29.0,64.0,73.0,36.0,75.0,105.0,70.0,103.0,13.0,38.0,57.0,54.0,23.0,30.0,83.0,137.0,60.0,43.0,66.0,126.0,37.0,75.0,126.0,127.0,148.0,140.0,103.0,43.0,81.0,20.0,123.0,0
2,87.0,7.0,63.0,24.0,14.0,63.0,17.0,164.0,91.0,37.0,129.0,26.0,56.0,69.0,48.0,81.0,93.0,11.0,17.0,76.0,34.0,56.0,148.0,17.0,56.0,42.0,21.0,106.0,74.0,19.0,25.0,97.0,18.0,129.0,109.0,68.0,63.0,19.0,11.0,22.0,103.0,41.0,37.0,27.0,106.0,29.0,29.0,23.0,64.0,14.0,45.0,157.0,18.0,109.0,83.0,109.0,36.0,125.0,21.0,84.0,0
3,24.0,98.0,33.0,21.0,58.0,50.0,64.0,91.0,42.0,38.0,28.0,146.0,78.0,34.0,27.0,30.0,51.0,28.0,70.0,29.0,77.0,117.0,100.0,69.0,20.0,36.0,89.0,26.0,63.0,13.0,40.0,43.0,44.0,35.0,44.0,55.0,96.0,241.0,73.0,56.0,38.0,54.0,76.0,38.0,61.0,63.0,56.0,28.0,30.0,12.0,24.0,86.0,28.0,71.0,86.0,44.0,42.0,51.0,69.0,87.0,0
4,10.0,118.0,22.0,29.0,38.0,30.0,61.0,51.0,34.0,16.0,64.0,77.0,32.0,39.0,26.0,33.0,39.0,28.0,54.0,22.0,53.0,62.0,73.0,32.0,49.0,48.0,54.0,12.0,46.0,24.0,48.0,32.0,28.0,18.0,42.0,25.0,39.0,46.0,48.0,45.0,26.0,56.0,31.0,41.0,54.0,45.0,46.0,19.0,17.0,28.0,21.0,54.0,5.0,48.0,73.0,49.0,36.0,21.0,31.0,67.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,70.0,198.0,35.0,196.0,118.0,110.0,132.0,38.0,65.0,27.0,62.0,63.0,128.0,116.0,103.0,123.0,115.0,195.0,156.0,118.0,75.0,38.0,44.0,118.0,145.0,130.0,82.0,36.0,79.0,141.0,130.0,82.0,100.0,66.0,44.0,115.0,87.0,164.0,272.0,117.0,150.0,250.0,178.0,75.0,68.0,135.0,281.0,100.0,72.0,125.0,103.0,62.0,11.0,30.0,64.0,38.0,152.0,72.0,214.0,60.0,4
196,1300.0,176.0,296.0,75.0,107.0,805.0,120.0,431.0,859.0,226.0,196.0,137.0,798.0,176.0,395.0,431.0,163.0,104.0,127.0,523.0,223.0,248.0,338.0,188.0,455.0,297.0,163.0,1448.0,906.0,93.0,190.0,937.0,107.0,780.0,841.0,982.0,765.0,230.0,344.0,190.0,1117.0,320.0,340.0,182.0,519.0,168.0,381.0,557.0,1726.0,102.0,799.0,451.0,225.0,360.0,276.0,193.0,165.0,833.0,111.0,308.0,4
197,25.0,206.0,105.0,44.0,68.0,106.0,125.0,319.0,110.0,99.0,103.0,162.0,209.0,62.0,32.0,116.0,47.0,41.0,101.0,90.0,195.0,236.0,317.0,38.0,28.0,129.0,182.0,13.0,187.0,60.0,151.0,129.0,61.0,102.0,180.0,143.0,176.0,138.0,99.0,134.0,79.0,98.0,70.0,150.0,277.0,109.0,33.0,39.0,97.0,44.0,31.0,299.0,23.0,266.0,303.0,104.0,81.0,106.0,100.0,289.0,4
198,0.0,8.0,1.0,1.0,9.0,6.0,11.0,4.0,7.0,4.0,1.0,3.0,7.0,5.0,3.0,2.0,4.0,3.0,12.0,2.0,2.0,2.0,2.0,12.0,0.0,8.0,1.0,0.0,3.0,4.0,7.0,5.0,7.0,7.0,1.0,2.0,2.0,9.0,11.0,6.0,2.0,9.0,3.0,0.0,4.0,10.0,9.0,3.0,3.0,10.0,8.0,5.0,0.0,2.0,1.0,2.0,4.0,2.0,8.0,7.0,4
